In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# !pip install nilearn
# !pip install giotto-tda

In [24]:
import os
import zipfile
import nibabel as nib
import numpy as np
from nilearn import image, masking
from nilearn.connectome import ConnectivityMeasure
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import PersistenceEntropy
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from nilearn.input_data import NiftiMasker

# # Set the directory for the dataset
# dataset_dir = "/content/drive/My Drive/data today"  # Update this path

# # Extract and save NIfTI files
# extracted_dir = "/content/extracted_data"
# os.makedirs(extracted_dir, exist_ok=True)

# for subject in [sub for sub in os.listdir(dataset_dir) if sub.startswith('sub')]:
#     subject_path = os.path.join(dataset_dir, subject)
#     for subfolder in ['anat', 'func']:
#         subfolder_path = os.path.join(subject_path, subfolder)
#         if os.path.exists(subfolder_path):
#             for zip_file in os.listdir(subfolder_path):
#                 if zip_file.endswith(".zip"):
#                     with zipfile.ZipFile(os.path.join(subfolder_path, zip_file), 'r') as zip_ref:
#                         zip_ref.extractall(os.path.join(extracted_dir, subject, subfolder))

# print("Extraction completed!")

In [ ]:
import os
import zipfile

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


dataset_dir = "/content/drive/MyDrive/data today"  

# List subjects (sub001 to sub114)
subjects = [f'sub-{i:03d}' for i in range(1, 115)]

# Extract and save NIfTI files
extracted_dir = "/content/extracted_data"
os.makedirs(extracted_dir, exist_ok=True)

for subject in subjects:
    subject_dir = os.path.join(dataset_dir, subject)
    anat_dir = os.path.join(subject_dir, 'anat')
    func_dir = os.path.join(subject_dir, 'func')

    # Process anat and func directories if they exist
    for folder in [anat_dir, func_dir]:
        if os.path.exists(folder):
            for zip_file in os.listdir(folder):
                if zip_file.endswith(".zip"):
                    with zipfile.ZipFile(os.path.join(folder, zip_file), 'r') as zip_ref:
                        extract_path = os.path.join(extracted_dir, subject)
                        os.makedirs(extract_path, exist_ok=True)
                        zip_ref.extractall(extract_path)
        else:
            print(f"Folder not found: {folder}. Skipping this folder.")

print("Extraction completed!")



In [ ]:
import os

def verify_extraction(extracted_dir, subjects):
    for subject in subjects:
        func_dir = os.path.join(extracted_dir, subject, 'func')

        # Check if the directory exists
        if os.path.exists(func_dir):
            # List all files in the func directory
            files_in_func = os.listdir(func_dir)
            print(f"Files in {func_dir}: {files_in_func}")
        else:
            print(f"Directory not found: {func_dir}")

# Call the function to verify extraction
verify_extraction(extracted_dir, subjects)

In [41]:
#This function is for connectivity matrix
def compute_dynamic_connectivity(subject_func_path):
    func_img = nib.load(subject_func_path)
    brain_mask = masking.compute_brain_mask(func_img)
    func_img = image.clean_img(func_img, standardize=True, mask_img=brain_mask)
    masker = NiftiMasker(mask_img=brain_mask, standardize=True)
    time_series = masker.fit_transform(func_img)
    connectivity_measure = ConnectivityMeasure(kind='correlation')
    connectivity_matrix = connectivity_measure.fit_transform([time_series])[0]
    return connectivity_matrix

In [42]:
# For Persistence entropy
def extract_persistence_entropy(connectivity_matrix):
    VR_persistence = VietorisRipsPersistence(homology_dimensions=[0, 1])
    persistence_diagram = VR_persistence.fit_transform([connectivity_matrix])
    entropy = PersistenceEntropy().fit_transform(persistence_diagram)
    return entropy

In [43]:
# Analyze connectivity matrix (Objective 1)
def analyze_connectivity_matrix(subjects):
    for subject in subjects:
        subject_func_path = os.path.join(extracted_dir, subject, 'func', f"{subject}_task-rest_bold.nii.gz")
        if os.path.exists(subject_func_path):
            connectivity_matrix = compute_dynamic_connectivity(subject_func_path)
            np.save(f"/content/connectivity_matrices/{subject}_connectivity.npy", connectivity_matrix)
            print(f"Connectivity matrix for {subject} saved.")

In [44]:
# Save Persistence Entropy (Objective 2)
def save_persistence_entropy(subjects):
    for subject in subjects:
        connectivity_matrix = np.load(f"/content/connectivity_matrices/{subject}_connectivity.npy")
        entropy = extract_persistence_entropy(connectivity_matrix)
        np.save(f"/content/persistence_entropy/{subject}_entropy.npy", entropy)
        print(f"Persistence entropy for {subject} saved.")

In [45]:
# Visualize TDA Features using PCA (Objective 3)
def visualize_tda_features(subjects):
    tda_features = []
    for subject in subjects:
        entropy = np.load(f"/content/persistence_entropy/{subject}_entropy.npy")
        tda_features.append(entropy)

    tda_features = np.array(tda_features)
    scaler = StandardScaler()
    tda_features_scaled = scaler.fit_transform(tda_features)
    pca = PCA(n_components=2)
    tda_features_pca = pca.fit_transform(tda_features_scaled)

    plt.scatter(tda_features_pca[:, 0], tda_features_pca[:, 1])
    plt.title("PCA of TDA Features")
    plt.show()

In [ ]:
# # Run each objective sequentially
def analyze_connectivity_matrix(subjects):
    os.makedirs("/content/connectivity_matrices", exist_ok=True)  

    for subject in subjects:
        
        subject_func_path = os.path.join(extracted_dir, subject, 'func', f"{subject}_task-rest_bold.nii.gz")


        print(f"Processing {subject}...")
        print(f"Expected functional file path: {subject_func_path}")

        if os.path.exists(subject_func_path):
            print(f"File found for {subject}, computing connectivity matrix...")
            connectivity_matrix = compute_dynamic_connectivity(subject_func_path)
            save_path = f"/content/connectivity_matrices/{subject}_connectivity.npy"
            np.save(save_path, connectivity_matrix)
            print(f"Connectivity matrix for {subject} saved at {save_path}.")
        else:
            print(f"File not found for {subject}: {subject_func_path}")

analyze_connectivity_matrix(subjects)

In [ ]:
save_persistence_entropy(subjects)